<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Objects_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, the aim is to create an algorithm to detect objects on the video frames

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import IPython

In [ ]:
IPython.display.clear_output()

In [2]:
# Generate synthetic data
def generate_synthetic_data(num_samples, image_size=(128, 128), num_objects_range=(5, 20)):
    data = []
    for _ in range(num_samples):
        num_objects = np.random.randint(*num_objects_range)
        image = np.zeros(image_size)
        for _ in range(num_objects):
            object_x = np.random.randint(0, image_size[1])
            object_y = np.random.randint(0, image_size[0])
            image[object_y, object_x] += 1
        data.append(image)
    return np.array(data)

In [4]:
# Create the density map estimation model
def create_density_map_model(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    density_map = tf.keras.layers.Conv2D(1, (1, 1), activation='relu', padding='same')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=density_map)
    return model

In [5]:
# Prepare the data
num_samples = 1000
image_size = (128, 128)
input_data = generate_synthetic_data(num_samples, image_size)
density_maps = input_data.reshape(num_samples, image_size[0], image_size[1], 1)

In [6]:
input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
# Create and compile the model
input_shape = (image_size[0], image_size[1], 1)
model = create_density_map_model(input_shape)
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# Train the model
model.fit(input_data, density_maps, batch_size=32, epochs=10, validation_split=0.2)

# Test the model on a new image
test_image = generate_synthetic_data(1, image_size)[0]
predicted_density_map = model.predict(test_image.reshape(1, image_size[0], image_size[1], 1))

# Perform object counting by summing up the density map values
predicted_count = np.sum(predicted_density_map)

print(f"True object count: {test_image.sum()}")
print(f"Predicted object count: {predicted_count}")